In [1]:
from keras.utils.io_utils import HDF5Matrix
import matplotlib.pyplot as plt 
import numpy as np
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers import Activation
from keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D
from keras.utils import to_categorical, multi_gpu_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [2]:
fileName = "C:/Users/m/data/augmentation_r2/data set.hdf5"
trainX = HDF5Matrix(fileName, 'trainX')
trainY = HDF5Matrix(fileName, 'trainY')
testX = HDF5Matrix(fileName, 'testX')
testY = HDF5Matrix(fileName, 'testY')

In [3]:
print('', trainX.shape, '\n', trainY.shape, '\n', testX.shape, '\n', testY.shape)

 (234504, 227, 227, 1) 
 (234504, 18) 
 (100512, 227, 227, 1) 
 (100512, 18)


In [4]:
model = Sequential()

model.add(Conv2D(48, (11, 11), strides=(4, 4), input_shape=(227, 227, 1), activation='relu'))
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

model.add(Conv2D(128, (5, 5), padding='same', activation='relu'))
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

model.add(Conv2D(192, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(192, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu'))

model.add(Dense(18))
model.add(Activation('softmax'))

model = multi_gpu_model(model, gpus=2)

In [5]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

In [6]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10)
fileName = 'C:/Users/m/data/augmentation_r2/weights.{epoch:.2f}-{val_loss:.2f}.hdf5'
checkPoint = ModelCheckpoint(fileName, monitor='val_loss', period=1)

In [7]:
model.fit(trainX, trainY, batch_size=200, shuffle="batch", epochs=100, validation_data=(testX, testY), callbacks=[earlyStopping, checkPoint])

Train on 234504 samples, validate on 100512 samples
Epoch 1/100
 19000/234504 [=>............................] - ETA: 2:59 - loss: 0.0000e+00 - acc: 0.5137

KeyboardInterrupt: 

In [ ]:
model_nonSoftmax = Sequential(model.layers[0:14])
model_nonSoftmax.save('model_nonSoftmax.h5')